In [1]:
!pip install torch torchvision torchaudio
!pip install scikit-image

In [2]:
import os
from PIL import Image
from tqdm import tqdm

def clean_corrupted_images(root_folder):
    removed = 0
    total = 0

    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff")):
                total += 1
                file_path = os.path.join(root, file)

                try:
                    img = Image.open(file_path)
                    img.verify()
                except Exception:
                    print(f"❌ Removing corrupted image: {file_path}")
                    removed += 1
                    os.remove(file_path)

    print("====================================")
    print(f"Checked: {total} images")
    print(f"Removed: {removed} corrupted images")
    print("====================================")

clean_corrupted_images("./ddata")

Checked: 112183 images
Removed: 0 corrupted images


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# === 1. Path to ddata folder (same directory as notebook) ===
path = "./ddata"

train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# === 3. Create datasets ===
train_dataset = datasets.ImageFolder(
    root=os.path.join(path, "train"),
    transform=train_transforms
)

test_dataset = datasets.ImageFolder(
    root=os.path.join(path, "test"),
    transform=test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

class_names = train_dataset.classes
num_classes = len(class_names)
print("Classes:", class_names)

Classes: ['fake', 'real']


In [4]:
device = torch.device("gpu" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [10]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),   

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2), 
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN(num_classes=num_classes).to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
          f"|| Test Loss: {val_loss:.4f} | Test Acc: {val_acc:.4f}")

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16384, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout

In [9]:
import os
import numpy as np
from torchvision import datasets, transforms
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

path = "./ddata"

train_dataset = datasets.ImageFolder(
    root=os.path.join(path, "train"),
    transform=train_transforms
)

test_dataset = datasets.ImageFolder(
    root=os.path.join(path, "test"),
    transform=test_transforms
)

class_names = train_dataset.classes
num_classes = len(class_names)
print("Classes:", class_names)

# =========================================
# 3. LBP feature extractor
# =========================================
P = 8
R = 1
n_bins = P + 2  # uniform LBP bins = 10

def extract_lbp_hist(tensor_img):
    """
    Input: image tensor after transforms (C,H,W)
    Output: normalized LBP histogram (10-d vector)
    """
    img = tensor_img.clone()
    img = img * 0.5 + 0.5

    np_img = img.permute(1, 2, 0).cpu().numpy()
    gray = (np_img * 255).astype("uint8")
    gray = gray.mean(axis=2)  # grayscale

    lbp = local_binary_pattern(gray, P, R, method="uniform")

    hist, _ = np.histogram(lbp.ravel(),
                           bins=np.arange(0, n_bins+1),
                           range=(0, n_bins))

    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return hist


def dataset_to_lbp(dataset):
    X_feats = []
    y_labels = []

    for tensor_img, label in dataset:
        feat = extract_lbp_hist(tensor_img)
        X_feats.append(feat)
        y_labels.append(label)

    X_feats = np.stack(X_feats, axis=0)
    y_labels = np.array(y_labels)
    return X_feats, y_labels

print("Extracting LBP features for TRAIN...")
X_train, y_train = dataset_to_lbp(train_dataset)
print("Train feature shape:", X_train.shape)

print("Extracting LBP features for TEST...")
X_test, y_test = dataset_to_lbp(test_dataset)
print("Test feature shape:", X_test.shape)

model = make_pipeline(
    StandardScaler(),
    SVC(kernel="rbf", C=1.0, gamma="scale")
)

print("Training SVM...")
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test  = model.predict(X_test)

print("="*50)
print(f"Train Accuracy: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"Test  Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print("="*50)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test, target_names=class_names))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))

Classes: ['fake', 'real']
Extracting LBP features for TRAIN...


/Users/mikew/miniconda3/lib/python3.11/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Train feature shape: (90407, 10)
Extracting LBP features for TEST...
Test feature shape: (21776, 10)
Training SVM...
Train Accuracy: 0.6435
Test  Accuracy: 0.6040

Classification Report:
              precision    recall  f1-score   support

        fake       0.59      0.59      0.59     10399
        real       0.62      0.62      0.62     11377

    accuracy                           0.60     21776
   macro avg       0.60      0.60      0.60     21776
weighted avg       0.60      0.60      0.60     21776


Confusion Matrix:
[[6087 4312]
 [4311 7066]]


In [5]:
def run_training(model, train_loader, test_loader, device, num_epochs=10, lr=1e-4,
                 add_noise_fn_train=None, add_noise_fn_test=None, tag="MODEL"):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    print(f"\n===== Training {tag} =====")
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(
            model, train_loader, optimizer, criterion, device, add_noise_fn_train
        )
        val_loss, val_acc = evaluate(
            model, test_loader, criterion, device, add_noise_fn_test
        )

        print(
            f"Epoch [{epoch+1}/{num_epochs}] "
            f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
            f"|| Test Loss: {val_loss:.4f} | Test Acc: {val_acc:.4f}"
        )
    return model

In [6]:
def build_resnet18(num_classes):
    resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    in_features = resnet.fc.in_features
    resnet.fc = nn.Linear(in_features, num_classes)
    return resnet


resnet18_model = build_resnet18(num_classes)
resnet18_model = run_training(
    resnet18_model,
    train_loader,
    test_loader,
    device,
    num_epochs=10,
    lr=1e-4,
    tag="ResNet18 Transfer Learning"
)


===== Training ResNet18 Transfer Learning =====
Epoch [1/10] Train Loss: 0.2106 | Train Acc: 0.9129 || Test Loss: 0.2546 | Test Acc: 0.9266


KeyboardInterrupt: 

In [7]:
class DCGANDiscriminator(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


gan_disc_model = DCGANDiscriminator(num_classes)
gan_disc_model = run_training(
    gan_disc_model,
    train_loader,
    test_loader,
    device,
    num_epochs=10,
    lr=2e-4,
    tag="GAN-style Discriminator"
)


===== Training GAN-style Discriminator =====


KeyboardInterrupt: 

In [ ]:
class DiffusionStyleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2), 
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), 
            nn.Flatten(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


T = 1000
betas = torch.linspace(1e-4, 0.02, T)
alphas = 1.0 - betas
alpha_cumprod = torch.cumprod(alphas, dim=0).to(device)

def make_diffusion_noise_fn(train=True):
    def add_noise(x):
        b = x.size(0)
        t = torch.randint(0, T, (b,), device=x.device)  # random timestep
        a_bar = alpha_cumprod[t].view(-1, 1, 1, 1)
        noise = torch.randn_like(x)
        noisy = torch.sqrt(a_bar) * x + torch.sqrt(1 - a_bar) * noise
        return noisy
    return add_noise

diffusion_model = DiffusionStyleCNN(num_classes)
diffusion_model = run_training(
    diffusion_model,
    train_loader,
    test_loader,
    device,
    num_epochs=10,
    lr=1e-3,
    add_noise_fn_train=make_diffusion_noise_fn(train=True),
    add_noise_fn_test=None,
    tag="Diffusion-style Noisy CNN"
)